In [34]:
# Import merged df
import pandas as pd
merged = pd.read_csv('../data/test/merged_small.csv')

display(merged.head())
print("Unique Video IDs:", merged["video_id"].nunique())

/var/folders/18/k3vb5tx15r921kvp0jfm94sr0000gn/T/ipykernel_65329/84259136.py:3: DtypeWarning: Columns (15,17) have mixed types. Specify dtype option on import or set low_memory=False.
  merged = pd.read_csv('../data/test/merged_small.csv')


,user_id,video_id,play_duration_x,video_duration_x,time,date_x,timestamp,watch_ratio,user_active_degree,is_lowactive_period,...,download_user_num,report_cnt,report_user_num,reduce_similar_cnt,reduce_similar_user_num,collect_cnt,collect_user_num,cancel_collect_cnt,cancel_collect_user_num,num_friends
0,14,148,4381,6067,2020-07-05 05:27:48.378,20200705.0,1.593898e+09,0.722103,full_active,0,...,4.523810,0.000000,0.000000,35.936508,34.777778,0.175,0.175,0.050,0.050,NaN
1,14,183,11635,6100,2020-07-05 05:28:00.057,20200705.0,1.593898e+09,1.907377,full_active,0,...,10.444444,0.047619,0.047619,51.809524,50.619048,0.250,0.250,0.200,0.200,NaN
2,14,3649,22422,10867,2020-07-05 05:29:09.479,20200705.0,1.593898e+09,2.063311,full_active,0,...,48.079365,0.095238,0.095238,52.666667,51.634921,0.175,0.175,0.325,0.325,NaN
3,14,5262,4479,7908,2020-07-05 05:30:43.285,20200705.0,1.593898e+09,0.566388,full_active,0,...,122.253968,0.238095,0.222222,399.730159,384.047619,14.400,14.325,5.350,5.350,NaN
4,14,8234,4602,11000,2020-07-05 05:35:43.459,20200705.0,1.593899e+09,0.418364,full_active,0,...,2.793651,0.015873,0.015873,23.365079,22.825397,0.075,0.075,0.000,0.000,NaN


Unique Video IDs: 3327


In [37]:
# Let's build a user-item matrix
user_item_matrix = merged.pivot_table(
    index="user_id", columns="video_id", values="watch_ratio", fill_value=0
)
display(user_item_matrix.head())

video_id,103,109,120,122,128,130,131,133,136,137,...,10430,10436,10457,10462,10500,10506,10519,10552,10589,10595
user_id,,,,,,,,,,,,,,,,,,,,,
14,0.429126,1.482039,0.728738,0.477810,0.439333,1.150245,1.816317,0.781122,1.554396,2.307486,...,0.348932,0.965220,1.787169,1.816010,0.150323,1.535256,2.757278,0.143556,1.273362,1.719201
19,0.624466,1.070684,1.006064,0.759092,0.882691,0.639313,0.670019,1.407319,0.874814,0.722665,...,0.642896,0.633833,0.586222,1.178295,0.000000,0.977297,1.266322,0.265038,0.928168,1.107873
21,1.415049,1.028840,1.809125,0.688823,0.588365,0.619549,0.818749,1.944596,1.015039,0.575723,...,0.896847,0.918930,0.602573,0.995887,1.173871,0.957399,1.148837,0.216699,1.210398,1.713792
23,0.169223,2.549891,0.247487,0.438669,0.114338,0.828292,0.038440,2.455882,1.128438,1.021400,...,0.577134,0.335534,5.304503,0.610346,0.185161,4.725427,0.338674,0.430445,2.225363,0.000000
24,0.345049,0.449337,0.802936,0.797411,1.875599,0.783867,2.104939,6.418434,0.228018,3.892566,...,0.884743,0.578658,0.300125,2.151558,2.311935,1.848424,0.388630,0.103633,0.547944,0.093900


In [35]:
# User-level aggregates
user_agg = merged.groupby("user_id").agg(
    {
        "watch_ratio": ["mean", "std"],
        "like_cnt": "mean",
        "comment_cnt": "mean",
        "share_cnt": "mean",
    }
)
user_agg.columns = [
    "user_watch_ratio_mean",
    "user_watch_ratio_std",
    "user_like_mean",
    "user_comment_mean",
    "user_share_mean",
]
user_agg = user_agg.reset_index()

# Item-level aggregates
item_agg = merged.groupby("video_id").agg(
    {
        "watch_ratio": ["mean", "std"],
        "like_cnt": "mean",
        "comment_cnt": "mean",
        "share_cnt": "mean",
    }
)
item_agg.columns = [
    "item_watch_ratio_mean",
    "item_watch_ratio_std",
    "item_like_mean",
    "item_comment_mean",
    "item_share_mean",
]
item_agg = item_agg.reset_index()

display(user_agg.head())
display(item_agg.head())

,user_id,user_watch_ratio_mean,user_watch_ratio_std,user_like_mean,user_comment_mean,user_share_mean
0,14,0.997809,1.065740,8777.771967,462.867516,313.745142
1,19,0.836359,0.416786,8749.320732,462.376657,315.346586
2,21,0.947653,0.947987,8807.145463,459.862650,316.165653
3,23,1.024486,1.492576,8587.254487,443.989067,311.376265
4,24,0.864112,1.628206,8752.385304,457.391942,315.575171


,video_id,item_watch_ratio_mean,item_watch_ratio_std,item_like_mean,item_comment_mean,item_share_mean
0,103,0.828615,0.542348,1549.269841,8.396825,99.952381
1,109,1.037994,0.573307,629.047619,11.904762,2.253968
2,120,1.103180,0.500161,2683.634921,61.000000,39.476190
3,122,0.723046,0.383198,169.301587,7.460317,3.873016
4,128,0.704681,0.359460,10863.793651,132.793651,62.587302


Let's create a Engagement Formula:
---
We will want to penalize a high votality on watch ratio std

$$
\text{engagement\_score}_i = 
w_1 \cdot \text{norm}(\text{item\_watch\_ratio\_mean}_i) +
w_2 \cdot \text{norm}(\text{item\_watch\_ratio\_std}_i) +
w_3 \cdot \text{norm}(\text{item\_like\_mean}_i) +
w_4 \cdot \text{norm}(\text{item\_comment\_mean}_i) +
w_5 \cdot \text{norm}(\text{item\_share\_mean}_i)
$$

Where:

- $\text{norm}(x)$ is the Min-Max normalized value of feature $x$ across all items.
- The weights used are:
    - $w_1 = 0.3$ for watch ratio mean
    - $w_2 = 0.1$ for watch ratio std
    - $w_3 = 0.3$ for like mean
    - $w_4 = 0.15$ for comment mean
    - $w_5 = 0.15$ for share mean

In [36]:
# LET'S CREATE A ENGAGEMENT SCORE

# Formula between watch ratio, like ratio, comment ratio, and share ratio
item_agg["engagement_score"] = (
    item_agg["item_watch_ratio_mean"] * 0.4
    + item_agg["item_like_mean"] * 0.3
    + item_agg["item_comment_mean"] * 0.2
    + item_agg["item_share_mean"] * 0.1
    - item_agg["item_watch_ratio_std"] * 0.5
)

# Sort by engagement score
item_agg = item_agg.sort_values("engagement_score", ascending=False)
display(item_agg.head())

,video_id,item_watch_ratio_mean,item_watch_ratio_std,item_like_mean,item_comment_mean,item_share_mean,engagement_score
3324,10552,0.330114,0.300510,513749.714286,16206.714286,6652.285714,158031.467505
2601,7814,0.098671,0.159311,264769.900000,8197.800000,1113.300000,81181.819813
3325,10589,0.882349,0.485715,229917.600000,27773.600000,1397.200000,74669.830082
1718,5027,1.197339,0.817618,221842.250000,7961.500000,919.750000,68237.020126
3317,10430,1.029696,2.211818,216483.916667,10757.333333,9463.250000,68042.272636
